## Семинар 13: "Обучение с подкреплением 1"

ФИО: Салихова Кария Илшатовна

###  FrozenLake


<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>



In [1]:
import gym
import numpy as np 

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

In [2]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [4]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:

SFFF
FHFH
FFFH
HFFG
observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [5]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 4
reward: 0.0
is game over?: False
printing new state:
  (Right)
SFFF
FHFH
FFFH
HFFG


In [3]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Попробуйте походить по замерзшему озеру, не упав в дырку. 
* Каждый шаг вы с вероятностью __0.5__ будете двигаться в выбранном направлении и с вероятностью __0.5__ в случайном.
* Если упадете, используйте __env.reset()__ чтобы перезапустить __env__

In [4]:
env.reset()

0

In [18]:
new_obs, reward, is_done, _ = env.step(action_to_i['left'])
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

new observation code: 5
reward: 0
is game over?: True
printing new state:
  (Left)
SFFF
FHFH
FFFH
HFFG


## Задание: 
Следуя шаблонам функций реализуйте алгоритм Policy iteration.
Протестируйте его с помощью функции __evaluate_policy__.
Попробуйте разные значение для gamma и сравните результаты.

In [5]:
def run_episode(env, policy, gamma = 1.0, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

In [6]:
def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [7]:
env.env.P[1][2]

[(0.3333333333333333, 5, 0.0, True),
 (0.3333333333333333, 2, 0.0, False),
 (0.3333333333333333, 1, 0.0, False)]

In [8]:
# Используя фиксированную value_function и жадную стратегию получите policy
def extract_policy(v, gamma = 1.0):
    policy = np.zeros(env.env.nS)
    q = np.zeros((env.env.nS, env.action_space.n))
    for s1 in env.env.P.keys():
        for s2 in env.env.P[s1].keys():
            for item in env.env.P[s1][s2]:
                q[s1, s2] += item[0] * (item[2] + gamma*v[item[1]])
        policy[s1] = np.argmax(q[s1, :])
    return policy

In [10]:
# Используя фиксированную policy с помощью сэмплирования получите оценку для value_function с точностью 1e-10
def compute_policy_v(env, policy, gamma=1.0):
    v = np.zeros(env.nS)
    while True:
        copy_v = v.copy()
        for s in range(env.nS):
            sums = []
            for item in env.P[s][policy[s]]:
                #v[s] += item[0] * (item[2] + gamma*v[item[1]])
                sums.append(item[0] * (item[2] + gamma*v[item[1]]))
            v[s] = np.sum(sums)
        if np.linalg.norm(v - copy_v) < 1e-10:
            break
    return v

In [11]:
def policy_iteration(env, gamma = 1.0):
    max_iterations = 10000
    policy = np.random.choice(np.arange(env.nA), env.nS)
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            break

        policy = new_policy
    return policy

In [13]:
gamma = [0.6, 0.8, 0.9, 0.99, 0.999, 0.9999, 1.0]
for g in gamma:
    env = gym.make("FrozenLake-v0")
    optimal_policy = policy_iteration(env.env, gamma=g)
    policy_score = evaluate_policy(env, optimal_policy, g, n=100)
    print('gamma: {}, score: {:f}'.format(g, policy_score))

gamma: 0.6, score: 0.002522
gamma: 0.8, score: 0.008596
gamma: 0.9, score: 0.056904
gamma: 0.99, score: 0.480439
gamma: 0.999, score: 0.664502
gamma: 0.9999, score: 0.737083
gamma: 1.0, score: 0.740000
